In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [3]:
class Scraper:
    '''
    This class takes one argument which is the url and scrapes the webpage into a data frame.
    ''' 
    def __init__(self,url):
        self.url = url
        self.club_name = self.url.split('/')[3]
        self.headers = {"User-Agent":"Mozilla/5.0"}
        self.response = requests.get(self.url, headers=self.headers)
        self.soup = BeautifulSoup(self.response.text, 'lxml')
        self.players = self.soup.find("table",{"class": "items"}).tbody.find_all('tr')
        '''
        Firstly, we will get the names
        '''
        self.Name = []
        for player in self.players:
            try:
                if player.find("td",{"class": "hauptlink"}).div.text in self.Name:
                    pass
                else:
                    self.Name.append(player.find("td",{"class": "hauptlink"}).div.text)
            except:
                pass
        self.club = pd.DataFrame(index = range(0,len(self.Name)),columns = ['Player','Foot','Market Value'])
        self.club['Player'] = self.Name
        '''
        Now it is time to download the foot
        '''
        self.frame = []
        self.Foot = []
        h = list(range(0,3*len(self.Name),3))
        f = list(range(1,3*len(self.Name),3))
        for player in self.players:
            self.frame.append(player.findNext("td",{"class": "zentriert"}).findNext("td",{"class": "zentriert"}).findNext("td",{"class": "zentriert"}).findNext("td",{"class": "zentriert"}).findNext("td",{"class": "zentriert"}).text)
        for idx,player in enumerate(self.frame):
            if idx in h:
                    self.Foot.append(player)

        self.club['Foot'] = self.Foot
        
        '''
        And finally the market value
        '''
        self.Market = []
        self.Value = []
        for player in self.players:
            self.Market.append(player.findNext("td",{"class": "rechts hauptlink"}).text)
        for idx,player in enumerate(self.Market):
            if idx in h:
                if "m" in player:
                    self.Value.append(int(float(player[1:len(player)-3])*1000))
                elif "Th" in player:
                    self.Value.append(int(player[1:len(player)-5]))
                else:
                    self.Value.append(0)

        self.club['Market Value'] = self.Value
        
    def dt(self):
        return self.club
    
    def nm(self):
        return self.club_name
        

In [4]:
a = Scraper('https://www.transfermarkt.com/sk-slavia-prague/kader/verein/62/plus/1/galerie/0?saison_id=2020').dt()
b = Scraper('https://www.transfermarkt.com/ac-sparta-praha/kader/verein/197/plus/1/galerie/0?saison_id=2020').dt()
c = Scraper('https://www.transfermarkt.com/fc-viktoria-plzen/kader/verein/941/plus/1/galerie/0?saison_id=2020').dt()
d = Scraper('https://www.transfermarkt.com/fk-mlada-boleslav/kader/verein/5546/plus/1/galerie/0?saison_id=2020').dt()
e = Scraper('https://www.transfermarkt.com/fc-slovan-liberec/kader/verein/697/plus/1/galerie/0?saison_id=2020').dt()
f = Scraper('https://www.transfermarkt.com/fc-banik-ostrava/kader/verein/377/plus/1/galerie/0?saison_id=2020').dt()
g = Scraper('https://www.transfermarkt.com/fk-jablonec/kader/verein/1322/plus/1/galerie/0?saison_id=2020').dt()
h = Scraper('https://www.transfermarkt.com/fc-bohemians-prag-1905/kader/verein/715/plus/1/galerie/0?saison_id=2020').dt()
i = Scraper('https://www.transfermarkt.com/fc-zbrojovka-brno/kader/verein/5225/plus/1/galerie/0?saison_id=2020').dt()

In [5]:
j = Scraper('https://www.transfermarkt.com/sk-sigma-olomouc/kader/verein/2311/plus/1/galerie/0?saison_id=2020').dt()
k = Scraper('https://www.transfermarkt.com/fk-pardubice/kader/verein/1496/plus/1/galerie/0?saison_id=2020').dt()
l = Scraper('https://www.transfermarkt.com/fk-teplice/kader/verein/814/plus/1/galerie/0?saison_id=2020').dt()
m = Scraper('https://www.transfermarkt.com/mfk-karvina/kader/verein/13726/plus/1/galerie/0?saison_id=2020').dt()
n = Scraper('https://www.transfermarkt.com/fc-fastav-zlin/kader/verein/5545/plus/1/galerie/0?saison_id=2020').dt()
o = Scraper('https://www.transfermarkt.com/1-fc-slovacko/kader/verein/5544/plus/1/galerie/0?saison_id=2020').dt()
p = Scraper('https://www.transfermarkt.com/sk-dynamo-ceske-budejovice/kader/verein/1321/plus/1/galerie/0?saison_id=2020').dt()
q = Scraper('https://www.transfermarkt.com/slezsky-fc-opava/kader/verein/479/plus/1/galerie/0?saison_id=2020').dt()
r = Scraper('https://www.transfermarkt.com/1-fk-pribram/kader/verein/2598/plus/1/galerie/0?saison_id=2020').dt()

In [6]:
clubs_F = [a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r]
Czech_L = pd.DataFrame()
for i in clubs_F:
    Czech_L = Czech_L.append(i, ignore_index = True, sort = False)
Czech_L

,Player,Foot,Market Value
0,Ondrej Kolar,right,7500
1,Jan Stejskal,-,300
2,Premysl Kovar,right,100
3,David Zima,-,3000
4,David Hovorka,right,2000
...,...,...,...
488,Jan Rezek,both,100
489,Tomas Docekal,right,150
490,Radek Voltr,right,150
491,Martin Jindracek,right,100


In [7]:
Czech_L = Czech_L[(Czech_L['Market Value'] != 0)]
Czech_L = Czech_L.drop_duplicates('Player',keep=False)
Czech_L

,Player,Foot,Market Value
0,Ondrej Kolar,right,7500
1,Jan Stejskal,-,300
2,Premysl Kovar,right,100
3,David Zima,-,3000
4,David Hovorka,right,2000
...,...,...,...
488,Jan Rezek,both,100
489,Tomas Docekal,right,150
490,Radek Voltr,right,150
491,Martin Jindracek,right,100


In [8]:
Czech_L.to_csv('Market Values',index=False)